In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('olist').getOrCreate()

25/03/02 05:34:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [1]:
hdfs_path = '/olist_ka_data/'

In [2]:
olist_customers_df = spark.read.csv("/olist_ka_data/olist_customers_dataset.csv", header=True, inferSchema=True)
olist_geolocation_df = spark.read.csv("/olist_ka_data/olist_geolocation_dataset.csv", header=True, inferSchema=True)
olist_order_items_df = spark.read.csv("/olist_ka_data/olist_order_items_dataset.csv", header=True, inferSchema=True)
olist_order_payments_df = spark.read.csv("/olist_ka_data/olist_order_payments_dataset.csv", header=True, inferSchema=True)
olist_order_reviews_df = spark.read.csv("/olist_ka_data/olist_order_reviews_dataset.csv", header=True, inferSchema=True)
olist_orders_df = spark.read.csv("/olist_ka_data/olist_orders_dataset.csv", header=True, inferSchema=True)
olist_products_df = spark.read.csv("/olist_ka_data/olist_products_dataset.csv", header=True, inferSchema=True)
olist_sellers_df = spark.read.csv("/olist_ka_data/olist_sellers_dataset.csv", header=True, inferSchema=True)
product_category_name_translation_df = spark.read.csv("/olist_ka_data/product_category_name_translation.csv", header=True, inferSchema=True)


In [3]:
from pyspark.sql.functions import *

In [4]:
def missing_values(df,df_name):
    print(f'Missing Values in {df_name} :')
    df.select([count(when(col(c).isNull(),1)).alias(c) for c in df.columns]).show()

In [5]:
missing_values(olist_customers_df,'customer Table')

Missing Values in customer Table :
+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



In [7]:
missing_values(olist_orders_df,'Orders Table')

Missing Values in Orders Table :


+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



In [8]:
missing_values(olist_order_payments_df,'payments')

Missing Values in payments :
+--------+------------------+------------+--------------------+-------------+
|order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------+------------------+------------+--------------------+-------------+
|       0|                 0|           0|                   0|            0|
+--------+------------------+------------+--------------------+-------------+



In [ ]:
numerical vs continous

# Handle Missing Values

1. Drop missing Values
2. Fill missing Values ( for numerical value ) 
3. Impute Missing Values ( mean, median , knn) (for continous data)

In [9]:
olist_orders_df.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [10]:
olist_orders_df_cleaned = olist_orders_df.fillna({'order_delivered_customer_date':'9999-12-31'})

In [11]:
olist_orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

No, OLTP doesn't accept and errors out -

In [12]:
olist_order_payments_df.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [ ]:
from pyspark.sql import functions as F

payment_df_with_null = olist_order_payments_df.withColumn(
    'payment_value', 
    F.when(F.col('payment_value') != 99.33, F.col('payment_value'))
     .otherwise(F.lit(None))
)


In [17]:
payment_df_with_null =olist_order_payments_df.withColumn('payment_value', \
                                                         when(col('payment_value') !=99.33,\
                                                                  col('payment_value')).otherwise(lit(None)))

In [18]:
payment_df_with_null.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         null|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [20]:
from pyspark.ml.feature import Imputer

In [24]:
imputer  = Imputer(inputCols=['payment_value'],outputCols=['payment_value_imputed']).setStrategy('median')

In [25]:
payment_df_imputed = imputer.fit(payment_df_with_null).transform(payment_df_with_null)

In [26]:
payment_df_imputed.show()

+--------------------+------------------+------------+--------------------+-------------+---------------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|payment_value_imputed|
+--------------------+------------------+------------+--------------------+-------------+---------------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|         null|                100.0|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|                24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|                65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|               107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|               128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|               

# Standardizing the format

In [6]:
def print_schema(df,df_name):
    print(f'Schema  in {df_name} :')
    df.printSchema()

In [8]:
print_schema(olist_orders_df,'orders')

Schema  in orders :
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [9]:
print_schema(olist_customers_df,'customers')

Schema  in customers :
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [10]:
print_schema(olist_order_payments_df,'payments')

Schema  in payments :
root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)



In [12]:
olist_orders_df.show(5)

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [13]:
olist_orders_df_cleaned = olist_orders_df.withColumn('order_purchase_timestamp',to_date(col('order_purchase_timestamp')))

In [14]:
print_schema(olist_orders_df_cleaned,'orders_cleaned')

Schema  in orders_cleaned :
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: date (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [15]:
olist_orders_df_cleaned.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|              2017-10-02|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|              2018-07-24|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [16]:
olist_order_payments_df.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|                   3|        51.84|
|1f78449c8

In [23]:
olist_order_payments_df_cleaned = olist_order_payments_df.withColumn('payment_type',when(col('payment_type')=='boleto','Bank Transfer')
                                                                     .when(col('payment_type')=='credit_card','Credit Card').otherwise('other'))
        

In [24]:
olist_order_payments_df_cleaned.show()

+--------------------+------------------+-------------+--------------------+-------------+
|            order_id|payment_sequential| payment_type|payment_installments|payment_value|
+--------------------+------------------+-------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1|  Credit Card|                   8|        99.33|
|a9810da82917af2d9...|                 1|  Credit Card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1|  Credit Card|                   1|        65.71|
|ba78997921bbcdc13...|                 1|  Credit Card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1|  Credit Card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1|  Credit Card|                   2|        96.12|
|771ee386b001f0620...|                 1|  Credit Card|                   1|        81.16|
|3d7239c394a212faa...|                 1|  Credit Card|                   3|        51.84|

# HW - Correct the above

In [28]:
print_schema(olist_customers_df,'customers')

Schema  in customers :
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [25]:
olist_customers_df.show()

+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|            

In [29]:
olist_customers_df_cleaned = olist_customers_df.withColumn('customer_zip_code_prefix',col('customer_zip_code_prefix').cast('string'))

# Remove Duplicate

In [30]:
customers_df_cleaned = olist_customers_df.dropDuplicates(['customer_id'])

# Data Transformation

In [31]:
order_with_details = olist_orders_df.join(olist_order_items_df,'order_id','left')\
.join(olist_order_payments_df,'order_id','left')\
.join(olist_customers_df,'customer_id','left')

In [32]:
order_with_details.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------+--------------------+--------------------+-------------------+------+-------------+------------------+------------+--------------------+-------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|            order_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|payment_sequential|payment_type|payment_installments|payment_value|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------+------------------------+-------------------+-------------------------

In [33]:
print_schema(order_with_details,'order_details')

Schema  in order_details :
root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: 

In [42]:
order_with_total_values = order_with_details.groupBy('order_id')\
.agg(sum('payment_value').alias('total_amount')).orderBy('total_amount',ascending=False)

In [43]:
order_with_total_values.show()

+--------------------+------------------+
|            order_id|      total_amount|
+--------------------+------------------+
|03caa2c082116e1d3...|         109312.64|
|ab14fdcfbe524636d...| 45256.00000000001|
|1b15974a0141d54e3...|44048.000000000015|
|2cc9089445046817a...|          36489.24|
|e8fa22c3673b1dd17...|30185.999999999993|
|736e1922ae60d0d6a...|          29099.52|
|9aec4e1ae90b23c7b...|           22346.6|
|71dab1155600756af...|21874.049999999996|
|912343626f370ead5...|          19457.04|
|4412d97cb2093633a...|          19174.38|
|9f5054bd9a3c71702...|18667.000000000004|
|428a2f660dc84138d...|          18384.75|
|3a213fcdfe7d98be7...|          17786.88|
|f80549a97eb203e15...|           17671.0|
|cf4659487be50c0c3...|          17069.76|
|be382a9e1ed251281...|16313.600000000002|
|637617b3ffe9e2f7a...|14963.639999999998|
|c52c7fbe316b5b9d5...|14577.569999999998|
|f60ce04ff8060152c...|14401.000000000002|
|73c8ab38f07dc9438...|14196.280000000004|
+--------------------+------------

In [44]:
olist_order_items_df.show(5)

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30|199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18|12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51|199.9|        18.14|
+--------------------+-------------+------------

In [45]:
quantiles = olist_order_items_df.approxQuantile('price',[0.01,0.99],0.0)

In [47]:
olist_order_items_df.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            112650|
|   mean|120.65373901471354|
| stddev|183.63392805026012|
|    min|              0.85|
|    25%|              39.9|
|    50%|             74.99|
|    75%|             134.9|
|    max|            6735.0|
+-------+------------------+



In [48]:
low_cutoff,high_cutoff = quantiles[0],quantiles[1]

In [51]:
order_item_df_filtered = olist_order_items_df.filter((col('price')>=low_cutoff) & (col('price')<=high_cutoff))

In [53]:
order_item_df_filtered.select('price').summary().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            110453|
|   mean|108.49213068006871|
| stddev|112.87303173792675|
|    min|              9.99|
|    25%|             39.99|
|    50%|              74.9|
|    75%|             130.0|
|    max|             890.0|
+-------+------------------+



In [54]:
olist_products_df.show(5)

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

In [55]:
products_cleaned = olist_products_df.withColumn('product_size_category',
                                                when(col('product_weight_g')<500,'small')\
                                                .when(col('product_weight_g').between(500,2000),'medium')
                                                .otherwise('large')
                                               )

In [57]:
products_cleaned.select('product_weight_g','product_size_category').show(5)

+----------------+---------------------+
|product_weight_g|product_size_category|
+----------------+---------------------+
|             225|                small|
|            1000|               medium|
|             154|                small|
|             371|                small|
|             625|               medium|
+----------------+---------------------+
only showing top 5 rows



In [58]:
order_with_details.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 

In [59]:
products_cleaned.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)
 |-- product_size_category: string (nullable = false)

